## Trying to Use CausalNex with Bayesian Network using StructureModel 

In [3]:
import pandas as pd
import numpy as np
from causalnex.structure import StructureModel
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from causalnex.network import BayesianNetwork
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

e:\Apps\Anaconda\envs\gym-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
# Set a random seed for reproducibility
np.random.seed(42)

# Generate data based on the logic table
# Here, 'Texture' directly determines 'Movability'
data_size = 2  # Define the size of your dataset
texture = np.random.randint(0, 2, data_size)  # Randomly generate 0s or 1s for 'Texture'

# 'Movability' directly depends on 'Texture': if 'Texture' is 0, 'Movability' is 0; if 'Texture' is 1, 'Movability' is 1
movability = np.copy(texture)  # Make 'Movability' identical to 'Texture'
# movability = np.random.randint(0,2,data_size)  # Randomly generate 1s 
friction = np.copy(texture)

# Create a DataFrame
df = pd.DataFrame({
    'Texture': texture,
    'Friction':friction,
    'Movability': movability
})

# Display the first few rows of the DataFrame
print(df)

   Texture  Friction  Movability
0        0         0           0
1        1         1           1


In [47]:
# To define the Model Manually
# sm = StructureModel()
# sm.add_edge("Texture", "Movability")
# sm.add_edge("Texture", "Friction")
# sm.add_edge("Friction", "Movability")

In [48]:
# To do autolearning from pandas 
from causalnex.structure.notears import from_pandas_lasso,from_pandas
sm = from_pandas_lasso(df,beta=0.001,w_threshold=0.1)
# sm = from_pandas(df,w_threshold=0.1)

e:\Apps\Anaconda\envs\gym-env\lib\site-packages\causalnex\structure\notears.py:514: RuntimeWarning: invalid value encountered in multiply
  + (rho * (np.trace(exp_hdmrd) - d) + alpha) * exp_hdmrd.T * wmat * 2


In [49]:
from IPython.display import Image
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

viz = plot_structure(
    sm,
    graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
    prog='fdp'
)
viz.draw('causal_graph.png', format='png')

e:\Apps\Anaconda\envs\gym-env\lib\site-packages\pygraphviz\agraph.py:1405: RuntimeWarning: Warning: node 'Texture', graph '' size too small for label

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


In [50]:
bn = BayesianNetwork(sm)
bn = bn.fit_node_states(df)
bn = bn.fit_cpds(df, method="BayesianEstimator", bayes_prior="K2")

e:\Apps\Anaconda\envs\gym-env\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


In [22]:
# Predicting the probabilities
predictions = bn.predict_probability(df, 'Movability')
print(predictions.head())

   Movability_0  Movability_1
0      0.978261      0.021739
1      0.017241      0.982759
2      0.978261      0.021739
3      0.978261      0.021739
4      0.978261      0.021739


In [23]:
binary_predictions = predictions['Movability_1'] > 0.5  # Adjust based on actual column names in predictions
print(binary_predictions.head())

0    False
1     True
2    False
3    False
4    False
Name: Movability_1, dtype: bool


In [45]:
binary_predictions.shape

(2,)

In [46]:
df.shape

(2, 3)

In [50]:
ab = pd.concat([df,binary_predictions],axis=1,ignore_index=False)
ab.head()

,Texture,Friction,Movability,Movability_1
0,0,0,1,True
1,1,1,1,True
